# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

In [5]:
print(api_url)
print(data)

https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448
{'avgTotalVolume': 95656763, 'calculationPrice': 'tops', 'change': -1.49, 'changePercent': -0.00924, 'close': 0, 'closeSource': 'falicfoi', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 164.4, 'iexAskSize': 105, 'iexBidPrice': 160.83, 'iexBidSize': 309, 'iexClose': 164.75, 'iexCloseTime': 1701130321582, 'iexLastUpdated': 1655006268247, 'iexMarketPercent': 0.01848594499141161, 'iexOpen': 161.98, 'iexOpenTime': 1664700169774, 'iexRealtimePrice': 164.57, 'iexRealtimeSize': 102, 'iexVolume': 419613, 'lastTradeTime': 1657766149850, 'latestPrice': 160.36, 'latestSource': 'IEX real time price', 'latestTime': '11:13:49 AM', 'latestUpdate': 1703631951101, 'latestVolum

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price)
print(market_cap)

160.36
2642022187619


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)

In [8]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
    ),
    
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,160.36,2642022187619,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = my_columns
        ),
    
    ignore_index=True
    )

{'avgTotalVolume': 97161351, 'calculationPrice': 'tops', 'change': -1.24, 'changePercent': -0.00781, 'close': 0, 'closeSource': 'oafcilfi', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 160.83, 'iexAskSize': 504, 'iexBidPrice': 157.66, 'iexBidSize': 4269, 'iexClose': 162.77, 'iexCloseTime': 1720204182051, 'iexLastUpdated': 1714565917357, 'iexMarketPercent': 0.018138625036406383, 'iexOpen': 163.56, 'iexOpenTime': 1677553469349, 'iexRealtimePrice': 165.06, 'iexRealtimeSize': 6, 'iexVolume': 422830, 'lastTradeTime': 1722248730065, 'latestPrice': 165.64, 'latestSource': 'IEX real time price', 'latestTime': '11:16:17 AM', 'latestUpdate': 1685529298126, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2612005248555, 'oddLotDela

{'avgTotalVolume': 94129474, 'calculationPrice': 'tops', 'change': -1.25, 'changePercent': -0.00803, 'close': 0, 'closeSource': 'lifafoci', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 164.51, 'iexAskSize': 105, 'iexBidPrice': 159.99, 'iexBidSize': 4401, 'iexClose': 158.9, 'iexCloseTime': 1682554004658, 'iexLastUpdated': 1657193802827, 'iexMarketPercent': 0.018560977403135014, 'iexOpen': 164.29, 'iexOpenTime': 1712755611983, 'iexRealtimePrice': 162.89, 'iexRealtimeSize': 6, 'iexVolume': 428383, 'lastTradeTime': 1654738644855, 'latestPrice': 164.54, 'latestSource': 'IEX real time price', 'latestTime': '11:16:17 AM', 'latestUpdate': 1705302812553, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2653122131000, 'oddLotDelay

{'avgTotalVolume': 97067639, 'calculationPrice': 'tops', 'change': -1.23, 'changePercent': -0.00799, 'close': 0, 'closeSource': 'aiflfcoi', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 167, 'iexAskSize': 109, 'iexBidPrice': 164.91, 'iexBidSize': 220, 'iexClose': 165.06, 'iexCloseTime': 1669433704199, 'iexLastUpdated': 1706886911130, 'iexMarketPercent': 0.018311474866027674, 'iexOpen': 161.41, 'iexOpenTime': 1650278361709, 'iexRealtimePrice': 164.5, 'iexRealtimeSize': 6, 'iexVolume': 425441, 'lastTradeTime': 1695359907719, 'latestPrice': 160.38, 'latestSource': 'IEX real time price', 'latestTime': '11:16:17 AM', 'latestUpdate': 1668808590830, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2703781697148, 'oddLotDelayedPr

{'avgTotalVolume': 97943748, 'calculationPrice': 'tops', 'change': -1.25, 'changePercent': -0.00788, 'close': 0, 'closeSource': 'liiffcoa', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 162.7, 'iexAskSize': 202, 'iexBidPrice': 162.32, 'iexBidSize': 4311, 'iexClose': 162.38, 'iexCloseTime': 1650685099536, 'iexLastUpdated': 1719744239974, 'iexMarketPercent': 0.0184321432484427, 'iexOpen': 162.74, 'iexOpenTime': 1656931807983, 'iexRealtimePrice': 160, 'iexRealtimeSize': 6, 'iexVolume': 421220, 'lastTradeTime': 1650411928798, 'latestPrice': 166.09, 'latestSource': 'IEX real time price', 'latestTime': '11:16:17 AM', 'latestUpdate': 1674340134255, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2704779151385, 'oddLotDelayedPri

{'avgTotalVolume': 94643433, 'calculationPrice': 'tops', 'change': -1.24, 'changePercent': -0.0078, 'close': 0, 'closeSource': 'offcaiil', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 159.97, 'iexAskSize': 104, 'iexBidPrice': 163.36, 'iexBidSize': 4335, 'iexClose': 158.9, 'iexCloseTime': 1693138100219, 'iexLastUpdated': 1687171390829, 'iexMarketPercent': 0.01877870200409972, 'iexOpen': 163.31, 'iexOpenTime': 1718337929595, 'iexRealtimePrice': 159.67, 'iexRealtimeSize': 6, 'iexVolume': 415583, 'lastTradeTime': 1676771878671, 'latestPrice': 158.51, 'latestSource': 'IEX real time price', 'latestTime': '11:16:17 AM', 'latestUpdate': 1713368636109, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2587128264987, 'oddLotDelayed

...

{'avgTotalVolume': 96702471, 'calculationPrice': 'tops', 'change': -1.24, 'changePercent': -0.00749, 'close': 0, 'closeSource': 'olaffcii', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 165.4, 'iexAskSize': 408, 'iexBidPrice': 163.41, 'iexBidSize': 4294, 'iexClose': 160.89, 'iexCloseTime': 1697426209780, 'iexLastUpdated': 1709102671629, 'iexMarketPercent': 0.018167699052497134, 'iexOpen': 164.34, 'iexOpenTime': 1650205364855, 'iexRealtimePrice': 165.73, 'iexRealtimeSize': 9, 'iexVolume': 428050, 'lastTradeTime': 1665485713481, 'latestPrice': 165.21, 'latestSource': 'IEX real time price', 'latestTime': '11:16:56 AM', 'latestUpdate': 1691637612408, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2688419400280, 'oddLotDelay

{'avgTotalVolume': 94901238, 'calculationPrice': 'tops', 'change': -1.23, 'changePercent': -0.00772, 'close': 0, 'closeSource': 'ifoafcil', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 161, 'iexAskSize': 103, 'iexBidPrice': 165.5, 'iexBidSize': 222, 'iexClose': 161.8, 'iexCloseTime': 1728306426729, 'iexLastUpdated': 1659272346595, 'iexMarketPercent': 0.01870971885240393, 'iexOpen': 162.23, 'iexOpenTime': 1674330926546, 'iexRealtimePrice': 165.72, 'iexRealtimeSize': 9, 'iexVolume': 435298, 'lastTradeTime': 1706715993116, 'latestPrice': 163.75, 'latestSource': 'IEX real time price', 'latestTime': '11:16:56 AM', 'latestUpdate': 1722550924139, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2616940933343, 'oddLotDelayedPric

{'avgTotalVolume': 97537954, 'calculationPrice': 'tops', 'change': -1.24, 'changePercent': -0.00771, 'close': 0, 'closeSource': 'ociilfaf', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 164.71, 'iexAskSize': 104, 'iexBidPrice': 164.8, 'iexBidSize': 102, 'iexClose': 158.68, 'iexCloseTime': 1708556671425, 'iexLastUpdated': 1720532015079, 'iexMarketPercent': 0.018791435942817264, 'iexOpen': 165.74, 'iexOpenTime': 1684617389222, 'iexRealtimePrice': 164.3, 'iexRealtimeSize': 9, 'iexVolume': 418593, 'lastTradeTime': 1654081743729, 'latestPrice': 163.52, 'latestSource': 'IEX real time price', 'latestTime': '11:16:56 AM', 'latestUpdate': 1702236472880, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2608675940987, 'oddLotDelayed

{'avgTotalVolume': 96156728, 'calculationPrice': 'tops', 'change': -1.22, 'changePercent': -0.00757, 'close': 0, 'closeSource': 'foiclfai', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 163, 'iexAskSize': 103, 'iexBidPrice': 160.35, 'iexBidSize': 4411, 'iexClose': 162.36, 'iexCloseTime': 1692535988257, 'iexLastUpdated': 1679430355250, 'iexMarketPercent': 0.018738108545787016, 'iexOpen': 164.15, 'iexOpenTime': 1677560964948, 'iexRealtimePrice': 163.58, 'iexRealtimeSize': 9, 'iexVolume': 429535, 'lastTradeTime': 1647674283670, 'latestPrice': 165.21, 'latestSource': 'IEX real time price', 'latestTime': '11:16:56 AM', 'latestUpdate': 1693722662692, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2709781503113, 'oddLotDelayed

KeyboardInterrupt: 

In [ ]:
final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'],100))

symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

symbol_groups
# final_dataframe = pd.DataFrame(columns = my_columns)

# for symbol_string in symbol_strings:
#     batch_api_url_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    
#     # some issue here, will check later
    
#     data = requests.get(batch_api_url_call).json()
#     for symbol in symbol_string.split(','):
#         final_dataframe = final_dataframe.append(
#         pd.Series(
#                 [
#                     symbol,
#                     data[symbol]['quote']['latestPrice'],
#                     data[symbol]['quote']['marketCap'],
#                     'N/A'
#                 ],
#                 index = my_columns
#             ),

#         ignore_index=True
#         )
# final_dataframe

In [ ]:
ssss

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [ ]:
portfolio_size = input("Enter the size of the portfolio : ")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number \nPlease try again : ")
    portfolio_size = input("Enter the size of the portfolio : ")
    val = float(portfolio_size)

In [ ]:
position_size = val/len(final_dataframe.index)

for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])

final_dataframe


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [ ]:
writer = pd.ExcelWriter('recommended trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color, 
        'border' : 1
    }
)

dollar_format = writer.book.add_format({
    'num_format':'$0.00',
    'font_color' : font_color ,
    'bg_color' : background_color, 
    'border' : 1
})

integer_format = writer.book.add_format({
    'num_format':'0',
    'font_color' : font_color ,
    'bg_color' : background_color, 
    'border' : 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [ ]:
column_formats = {
    'A': ['Ticker',string_format],
    'B': ['Stock Price',dollar_format],
    'C': ['Market Capitalization',dollar_format],
    'D': ['Number of Shares to Buy',integer_format]
}

for columns in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{columns}:{columns}',18,column_formats[columns][1])
    writer.sheets['Recommended Trades'].write(f'{columns}1',column_formats[columns][0],column_formats[columns][1])   

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()